In [1]:
from ngsolve import *
import netgen.gui
from netgen.geom2d import SplineGeometry
%gui tk

In [2]:
def SolveBVP():
    fes.Update()
    gfu.Update()
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)
    
l = []
def CalcError():
    space_flux.Update()#TODO funktionale von der Angabe einprogrammieren 
    gf_flux.Update()#TODO funktionale von der Angabe einprogrammieren 
    flux = grad(gfu)#TODO funktionale von der Angabe einprogrammieren 
    # interpolate finite element flux into H(div) space:
    gf_flux.Set (flux) #TODO funktionale von der Angabe einprogrammieren 
    
    # Gradient-recovery error estimator
    err = (flux-gf_flux)*(flux-gf_flux)*lam
    elerr = Integrate (err, mesh, VOL, element_wise=True) #todo über bnd_integration integrieren
    maxerr = max(elerr)
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    print ("maxerr = ", maxerr)

    for el in mesh.Elements():
        mesh.SetRefinementFlag(el, elerr[el.nr] > 0.25*maxerr)

In [3]:
#      3---------------------------2
#      |                           |
#      |                     (1)   |
#      |                           |
#      |                           |
#      |  7----6     11----10      |
#      |  | (2)|      |    |       |
#      |  |    |      | (3)|       |
#      |  4----5      8----9       |
#      |                           |
#      |                           |
#      |                           |
#      0---------------------------1
#

def MakeGeometry():
    geometry = SplineGeometry()
    
    # point coordinates ...
    pnts = [ (0,0), (1,0), (1,1), (0,1), 
             (0.2,0.45), (0.3,0.45), (0.3,0.55), (0.2,0.55),
             (0.7,0.45), (0.8,0.45), (0.8,0.55), (0.7,0.55)]
    p = [geometry.AppendPoint(*p) for p in pnts]
        
    lines = [[p[0], p[1], 1, 0, "bnd_outer"],
             [p[1], p[2], 1, 0, "bnd_outer"],
             [p[2], p[3], 1, 0, "bnd_outer"],
             [p[3], p[0], 1, 0, "bnd_outer"],
             [p[4], p[5], 2, 1, "bnd_inner"],
             [p[5], p[6], 2, 1, "bnd_inner"],
             [p[6], p[7], 2, 1, "bnd_inner"],
             [p[7], p[4], 2, 1, "bnd_inner"],
             [p[8], p[9], 3, 1, "bnd_integration"],
             [p[9], p[10], 3, 1, "bnd_integration"],
             [p[10], p[11], 3, 1, "bnd_integration"],
             [p[11], p[8], 3, 1, "bnd_integration"]]
    
    
    for start_point, end_point, ld, rd, name in lines:
        geometry.Append(["line", start_point, end_point], leftdomain=ld, rightdomain=rd ,bc=name) 

    geometry.SetMaterial(1,'outer')
    geometry.SetMaterial(3,'outer')
    geometry.SetMaterial(2,'inner')
    
    
    return geometry

In [4]:
geo = MakeGeometry()
mesh = Mesh(geo.GenerateMesh (maxh=1))
Draw(mesh)

In [5]:
fes = H1(mesh, order=3, dirichlet='bnd_outer')
u = fes.TrialFunction()
v = fes.TestFunction()

lam_val = {'inner':100, 'outer':0}
lam = CoefficientFunction([lam_val[x] for x in mesh.GetMaterials()])

a = BilinearForm(fes, symmetric=False)
a += SymbolicBFI(grad(u)*grad(v))


# heat-source in sub-domain 3
f = LinearForm(fes)
f += SymbolicLFI(lam*v)

c = Preconditioner(a, type="multigrid", inverse = "sparsecholesky")

In [6]:
gfu = GridFunction(fes)
Draw (gfu)

# finite element space and gridfunction to represent
# the heatflux:
space_flux = HDiv(mesh, order=2)
gf_flux = GridFunction(space_flux, "flux")
Draw(gf_flux)

In [7]:
with TaskManager():
    while fes.ndof < 1000:  
        SolveBVP()
        CalcError()
        mesh.Refine()
SolveBVP()

maxerr =  0.01212734986054724
maxerr =  0.0009387635268524046
maxerr =  5.0032373121924353e-05
maxerr =  3.1029575294639574e-06
